In [1]:
#Import Necessary Packages
import numpy as np
import pandas as pd

#Import Dataset
data = pd.read_csv('fifa21_training.csv')
data.dropna(inplace=True)
data.head()

,Name,Age,Nationality,Club,Default Position,Height,Weight,Value,Attacking,Crossing,...,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,€525K,258,54,...,High,Medium,1 ★,69,51,63,63,51,60,64
1,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,€8.5M,365,66,...,High,Low,2 ★,83,75,68,82,33,71,77
2,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,€9M,336,73,...,High,Medium,2 ★,80,77,78,86,27,56,80
3,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,€275K,242,44,...,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,€725K,249,49,...,Low,Medium,1 ★,66,44,60,64,60,66,65


In [2]:
#Change Notebook Settings
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 70)

In [3]:
#Check dtypes
data['Default Position'].unique()

array(['CM', 'ST', 'CAM', 'CDM', 'LM', 'RB', 'CB', 'RM', 'GK', 'LB', 'RW',
       'LWB', 'LW', 'RWB', 'CF'], dtype=object)

In [24]:
data['Value'] = [x.strip("€") for x in data['Value']]
data['Value'] = [x.strip("M") for x in data['Value']]
data['Value'] = [x.strip("K") for x in data['Value']]
data['Value'] = pd.to_numeric(data['Value'])
data.head()

,Name,Age,Nationality,Club,Default Position,Height,Weight,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,525.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,64
1,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,8.5,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,77
2,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,9.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,80
3,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,275.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,725.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,65


In [18]:
#Define a Function for Cleaning 
def clean(x):
    
    data = x
    
    #Drop Nationality and Clubs Columns
    data = data.drop(['Nationality', 'Club'], axis = 1)
    
    #Drop Players where GK is Default Position
    data.drop(data.loc[data['Default Position']=='GK'].index, inplace=True)
    
    #Stripping " from the Height row so that it can be split on '
    data['Height'] = data['Height'].str.strip("\"")
    
    #Creating a New Dataframe with the split columns
    new = data['Height'].str.split("\'", n = 1, expand = True)
    
    #Define Foot > Inches Conversion
    def ftin(x):
        return int (x)*12
    
    #Create a New Column in the Main DF with the converted Feet
    data['Inches2'] = list(map(ftin, new[0]))
    
    #Create a New Column in the Main DF with split inches
    data["Inches"]= new[1]
    
    # Sum the two and replace values in Height
    data['Height'] = data[['Inches', 'Inches2']].sum(axis = 1)
    
    #Define Inches to CM Conversion
    def toCM(x):
        return float (x)*2.54
    
    #Convert Corrected Heights into CM
    data['Height'] = list(map(toCM, data['Height']))
    
    #Drop Columns used for Manipulation
    data.drop(['Inches', 'Inches2'], axis=1, inplace=True)
    
    #reindex DataFrame
    
    data = data.reset_index()
    
    return data
    

In [19]:
clean(data)


,index,Name,Age,Default Position,Height,Weight,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,0,A. Pasche,26,CM,152.40,161lbs,525,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,64
1,1,Alan Carvalho,30,ST,182.88,159lbs,8.5,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,77
2,2,S. Giovinco,33,CAM,152.40,134lbs,9,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,80
3,3,J. Evans,22,CDM,152.40,152lbs,275,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,4,Y. Demoncy,23,CDM,152.40,150lbs,725,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12099,13694,A. Boljević,24,RM,182.88,174lbs,2.3,308,71,64,53,67,53.0,322,72,56.0,57,66,71,361,78,75,78.0,66,64.0,343,67,78.0,75,63,60,276,47,37.0,68.0,65.0,59,68.0,80,35,23,22.0,51,7,13,10,6,15,1741,373,5 ★,3★,High,Medium,1 ★,76,63,66,72,32,64,70
12100,13695,S. Aw,21,LB,152.40,143lbs,325,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3 ★,2★,High,Medium,1 ★,76,28,46,55,53,57,60
12101,13697,Carles Gil,27,RM,152.40,146lbs,8,332,76,72,34,79,71.0,373,77,76.0,65,77,78,351,64,65,74.0,73,75.0,329,63,60.0,83,57,66,306,41,40.0,75.0,81.0,69,75.0,118,36,43,39.0,58,12,15,16,9,6,1867,388,4 ★,4★,High,Medium,2 ★,65,69,78,77,39,60,76
12102,13698,J. Perch,34,CDM,152.40,176lbs,140,268,58,44,61,62,43.0,261,56,47.0,37,60,61,288,54,52,57.0,61,64.0,300,53,65.0,68,68,46,297,74,62.0,54.0,56.0,51,60.0,182,61,62,59.0,43,10,7,9,9,8,1639,346,3 ★,2★,Medium,Medium,1 ★,53,47,58,58,61,69,63


In [36]:
#Create Value Multiply Function

def valueMultiply(x):
        if (x) >= 100:
            return (x)*1000
        else:
            return (x)*1000000
            
#Apply the Function to the Value Column

data['Value'] = list(map(valueMultiply, data['Value']))

In [38]:
data.head()

,Name,Age,Nationality,Club,Default Position,Height,Weight,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,525000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,64
1,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,8500000.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,77
2,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,9000000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,80
3,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,275000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,725000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,65


In [ ]:
#attacking, skill, movement, power, mentality, defending
#remove goalkeepers

In [ ]:
data.dtypes

In [ ]:
X_num = data.select_dtypes(include = np.number)
X_cat = data.select_dtypes(include = np.object)

In [ ]:
manip = X_cat.drop(['Name', 'Weight', 'Value',], axis = 1)

In [ ]:
manip.head()

In [ ]:
#Encoding the Dummies using OneHot
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(manip)
encoded = encoder.transform(manip).toarray()
encoded

In [ ]:
X_num.head()

In [ ]:
#Create Value Multiply Function

def valueMultiply(x):
    for i in x:
        if i >= 100:
            i*1000
        else:
            i*1000000
            
#Apply the Function to the Value Column

data['Value'] = list(map(valueMultiply, data['Value']))